In [11]:
import duckdb as ddb
import pandas as pd

Establish a connection to the DuckDB database file and

con  # Display the connection object to verify it is active

In [12]:
con = ddb.connect("../air_quality.db")
con

In [13]:
df = con.query("select * from raw.air_quality where parameter in ('so2', 'pm10', 'pm25')").to_df()
#len(df)
df.head()

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime
0,225393,5079586,Damonsville-237689,2024-01-01 02:00:00,-25.625702,27.851825,pm10,µg/m³,115.332000,01,2024,2026-01-25 20:52:59.513809
1,225393,5079586,Damonsville-237689,2024-01-01 08:00:00,-25.625702,27.851825,pm10,µg/m³,42.030000,01,2024,2026-01-25 20:52:59.513809
2,225393,5079587,Damonsville-237689,2024-01-01 02:00:00,-25.625702,27.851825,pm25,µg/m³,64.063000,01,2024,2026-01-25 20:52:59.513809
3,225393,5079587,Damonsville-237689,2024-01-01 08:00:00,-25.625702,27.851825,pm25,µg/m³,16.585000,01,2024,2026-01-25 20:52:59.513809
4,225393,1303603,Damonsville-237689,2024-01-01 02:00:00,-25.625702,27.851825,so2,ppm,0.001821,01,2024,2026-01-25 20:52:59.513809


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12587 entries, 0 to 12586
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   location_id         12587 non-null  int64         
 1   sensors_id          12587 non-null  int64         
 2   location            12587 non-null  object        
 3   datetime            12587 non-null  datetime64[us]
 4   lat                 12587 non-null  float64       
 5   lon                 12587 non-null  float64       
 6   parameter           12587 non-null  object        
 7   units               12587 non-null  object        
 8   value               12587 non-null  float64       
 9   month               12587 non-null  object        
 10  year                12587 non-null  int64         
 11  ingestion_datetime  12587 non-null  datetime64[us]
dtypes: datetime64[us](2), float64(3), int64(3), object(4)
memory usage: 1.2+ MB


In [15]:
df.describe()

,location_id,sensors_id,datetime,lat,lon,value,year,ingestion_datetime
count,12587.000000,1.258700e+04,12587,12587.000000,12587.000000,12587.000000,12587.0,12587
mean,225081.594820,1.739676e+06,2024-02-06 11:41:09.118932,-22.250642,24.779755,28.179272,2024.0,2026-01-25 20:53:35.642285
min,220715.000000,1.286151e+06,2024-01-01 00:00:00,-29.637583,3.406815,0.000000,2024.0,2026-01-25 20:52:59.513809
25%,225393.000000,1.303424e+06,2024-01-20 00:00:00,-29.637583,26.241952,0.015568,2024.0,2026-01-25 20:53:13.081162
50%,225428.000000,1.303433e+06,2024-02-05 06:00:00,-29.138507,27.837328,12.186000,2024.0,2026-01-25 20:53:33.474394
75%,225443.000000,1.303602e+06,2024-02-24 03:00:00,-25.625702,30.337389,31.976500,2024.0,2026-01-25 20:53:55.957185
max,404479.000000,5.079587e+06,2024-04-01 00:00:00,9.041648,30.337389,875.556000,2024.0,2026-01-25 20:54:24.789994
std,8642.219888,1.208843e+06,NaN,13.678697,7.700305,49.057421,0.0,NaN


In [16]:
df.describe(include='O')

,location,parameter,units,month
count,12587,12587,12587,12587
unique,6,3,2,3
top,Pelonomi-NAQI-237739,pm25,µg/m³,01
freq,3460,5546,9083,5626


In [17]:
df[df.duplicated(subset=["location_id","datetime", "parameter", "units", "value", "month", "year", "ingestion_datetime"])]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [18]:
df.groupby(by=["parameter", "location_id"], as_index=False).count()

,parameter,location_id,sensors_id,location,datetime,lat,lon,units,value,month,year,ingestion_datetime
0,pm10,225393,728,728,728,728,728,728,728,728,728,728
1,pm10,225405,595,595,595,595,595,595,595,595,595,595
2,pm10,225428,1053,1053,1053,1053,1053,1053,1053,1053,1053,1053
3,pm10,225443,1161,1161,1161,1161,1161,1161,1161,1161,1161,1161
4,pm25,220715,1975,1975,1975,1975,1975,1975,1975,1975,1975,1975
5,pm25,225393,728,728,728,728,728,728,728,728,728,728
6,pm25,225405,595,595,595,595,595,595,595,595,595,595
7,pm25,225428,1053,1053,1053,1053,1053,1053,1053,1053,1053,1053
8,pm25,225443,1167,1167,1167,1167,1167,1167,1167,1167,1167,1167
9,pm25,404479,28,28,28,28,28,28,28,28,28,28


In [19]:
df[df['value'] < 0 ]

,location_id,sensors_id,location,datetime,lat,lon,parameter,units,value,month,year,ingestion_datetime


In [20]:
con.close()